In [ ]:

import pandas as pd
import numpy as np
import os, sys
import plotly.express as px
import h5py
from sklearn.ensemble import HistGradientBoostingRegressor
from scipy import stats
# pvlib imports
import pvlib
from pvlib.irradiance import complete_irradiance
from pvlib.location import Location

from darts.utils.missing_values import extract_subseries
from darts import TimeSeries

import zipfile
import requests

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoDataFrame

import warnings
warnings.filterwarnings('ignore')


In [ ]:

# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"

In [3]:
# Setting up data paths and folder structure

raw_data_path = os.path.join(os.getcwd(),'data','raw_data')
if not os.path.exists(raw_data_path):
    os.makedirs(raw_data_path)
    print("Created directory: " + raw_data_path)

raw_data_subfolders = ['load', 'pv', 'weather', 'imbalance']

for subfolder in raw_data_subfolders:
    subfolder_path = os.path.join(raw_data_path, subfolder)
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
        print("Created directory: " + subfolder_path)

clean_data_path = os.path.join(os.getcwd(),'data','clean_data')
if not os.path.exists(clean_data_path):
    os.makedirs(clean_data_path)
    print("Created directory: " + clean_data_path)




In [4]:
# Import custom functions
from utils.utils import *

# Data Imports & Preleminary Cleaning for the Project 

## &#x2460; Energy Community Load Data - Germany

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.5642902.svg)](https://doi.org/10.5281/zenodo.5642902) 

Note: Due to their size we have not included the datasets in the repo, but the above link will allow you to download them, np.

We have downloaded the load data for [2018,2019,2020] in [15 minute] resolution:

* 2018_data_15min.zip, 2019_data_15min.zip, 2020_data_15min.zip

The goal here is to import them, select the useful data, impute missing data where plausible and aggregate to one community for both temporal resolutions

In [ ]:

community_members = [ # these are the households with reliable data for the considered duration
            'SFH3', 'SFH4', 'SFH5', 'SFH9', 'SFH10',
            'SFH12', 'SFH16','SFH18','SFH19', 'SFH21',
            'SFH22', 'SFH23', 'SFH27', 'SFH28', 'SFH29',
            'SFH30', 'SFH31','SFH32', 'SFH36', 'SFH38'
            ]

resolution = '15min' # this is the resolution of the data we are considering, we will resample to 15min later

store = pd.HDFStore(os.path.join(raw_data_path, 'load', "df_load.h5"))
dfs_load = []
for year in [2018,
             2019,
            2020
             ]:


    hdf5_file = f"data/raw_data/load/{year}_data_{resolution}.hdf5"

    f = h5py.File(hdf5_file)
    group_no_pv = f["NO_PV"] #Only regard those profiles that are not mixed with PV generation
    dfs = {}

    for member in community_members:
        table = f["NO_PV"][member]["HOUSEHOLD"]["table"][:]
        df = pd.DataFrame(table).dropna().set_index("index")[["P_TOT"]]
        df.index = pd.to_datetime(df.index, unit = "s")
        dfs[member] = df
        
    df_load = pd.concat(dfs, axis=1)
    dfs_load.append(df_load)

df_load_individual_households = pd.concat(dfs_load, axis=0).droplevel(1, axis=1)
df_load_total = df_load_individual_households.sum(axis=1).to_frame('community_load')

store.put(f'{resolution}/per_household_load', df_load_individual_households, format='table')
store.put(f'{resolution}/community_load', df_load_total, format='table')

store.close()   

## &#x2461; PV Power Data - Netherlands

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.6906504.svg)](https://doi.org/10.5281/zenodo.6906504)

We have downloaded the 'filtered_pv_power_measurements_ac.csv' file

Note: While meta data is available for download, exact locations of individual PV systems are not included.

Note: Due to their size we have not included the datasets in the repo, but the above link will allow you to download them, and execute the notebook as long as the paths are adjusted.

### Power Data

In [ ]:
df_pv = pd.read_csv(os.path.join(os.getcwd(), 'data', 'raw_data', 'pv', 'filtered_pv_power_measurements_ac.csv'), index_col=0, parse_dates=True)

df_pv.head()


In [ ]:
# sort the dataframe columns by the number of NaNs and select the 20 with the least NaNs
top_systems = df_pv.isna().sum().sort_values().index[:20]

df_pv_filtered_1 = df_pv[top_systems]

In [ ]:
df_pv_filtered_2, days_with_nans_list = drop_days_with_nans_advances(df_pv_filtered_1, 0.9)

df_pv_filtered_2.head()


In [ ]:
df_droped_days = dropped_days_plotted(df_pv_filtered_1, days_with_nans_list)

In [ ]:
px.line(df_pv_filtered_2.resample('6H').mean(), title="PV profiles of individual households")



In [ ]:
df_pv_filtered_2.to_csv(os.path.join(raw_data_path, 'pv', 'filtered_pv_power_measurements_ac_2.csv'))

In [ ]:
df_pv = pd.read_csv(os.path.join(raw_data_path, 'pv', 'filtered_pv_power_measurements_ac_2.csv'), index_col=0, parse_dates=True).resample('60min').mean()

### Meta Data

In [ ]:
df_meta = pd.read_csv(os.path.join(raw_data_path, 'pv', 'metadata.csv'), sep=';')

df_meta['latitude'] = (df_meta['north'] + df_meta['south']) / 2
df_meta['longitude'] = (df_meta['east'] + df_meta['west']) / 2

df_meta = df_meta[['ID', 'latitude', 'longitude', 'tilt', 'azimuth', 'estimated_dc_capacity']].fillna(method = 'pad')

# select those systems that are in the filtered data

df_meta_filtered = df_meta[df_meta['ID'].isin(df_pv_filtered_2.columns)]

df_meta_filtered.to_csv(os.path.join(raw_data_path, 'pv', 'metadata_with_lat_lng.csv'), index=False)



In [ ]:
df_meta_filtered

In [ ]:

# Lets visualize the coordinates of the PV systems on a map

# Read the shapefile of the Netherlands
netherlands = gpd.read_file("https://stacks.stanford.edu/file/druid:st293bj4601/data.zip")

# Select the city of Utrecht
utrecht = netherlands.iloc[[10]]

# Create a dataframe of the coordinates
df_coords = df_meta_filtered[['ID','latitude','longitude']].copy()
df_coords.set_index('ID', inplace=True)

# Create a list of points from the coordinates
geometry = [Point(xy) for xy in zip(df_coords['longitude'], df_coords['latitude'])]

# Create a GeoDataFrame from the coordinates
gdf = GeoDataFrame(df_coords, geometry=geometry)

# Plot the map of the Netherlands
plt.figure(figsize=(10, 10))
ax = utrecht.plot(color='lightgray')

# Plot the coordinates on the map
gdf.plot(ax=ax, color='red', marker='o', alpha=0.5)

# Add a title to the plot
plt.title('Coordinates of Utrecht')
# Show the plot
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoDataFrame
import contextily as ctx

# Read the shapefile of the Netherlands
netherlands = gpd.read_file("https://stacks.stanford.edu/file/druid:st293bj4601/data.zip")

# Select the city of Utrecht
utrecht = netherlands.iloc[[10]]

# Create a dataframe of the coordinates
df_coords = df_meta_filtered[['ID', 'latitude', 'longitude', 'tilt', 'azimuth', 'estimated_dc_capacity']].copy()
df_coords.set_index('ID', inplace=True)

# Create a list of points from the coordinates
geometry = [Point(xy) for xy in zip(df_coords['longitude'], df_coords['latitude'])]

# Create a GeoDataFrame from the coordinates
gdf = GeoDataFrame(df_coords, geometry=geometry, crs="EPSG:4326")  # Set initial CRS to WGS84

# Convert the GeoDataFrame to Web Mercator for contextily
gdf = gdf.to_crs(epsg=3857)

# Extract x and y coordinates for plotting
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

# Plot the map of the Netherlands
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
utrecht = utrecht.to_crs(epsg=3857)
utrecht.plot(ax=ax, color='lightgray')

# Plot the coordinates on the map with reduced marker size
sc = ax.scatter(gdf['x'], gdf['y'],
                s=gdf['estimated_dc_capacity']/10, cmap='viridis', alpha=0.6, c = "orange",
                edgecolor='k', linewidth=0.5)

# Add basemap
#ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)

# Add a title to the plot
plt.title('Location of Households in Utrecht')


# Annotate the points with azimuth
for idx, row in gdf.iterrows():
    ax.annotate(text=f"A:{row['azimuth']}°, \nT: {row['tilt']}°", xy=(row['x'], row['y']), 
                xytext=(3, 3), textcoords='offset points', fontsize=8, color='black')

# Show the plot
plt.show()


## &#x2462; Irradiance Data - Netherlands

Here we will use https://open-meteo.com/en/docs to get the relevant weather data.

In the paper we used historic forecast data from https://solcast.com/, which requires an account.


Note: We are using the central coordinates of all systems to get weather data for the general area.


In [ ]:
lat_avg = df_meta['latitude'].mean()
lng_avg = df_meta['longitude'].mean()
start_date = df_pv_filtered_2.index[0].strftime('%Y-%m-%d')
end_date = df_pv_filtered_2.index[-1].strftime('%Y-%m-%d')
variables =  ['temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'direct_radiation']
df_weather = get_weather_data(lat_avg, lng_avg, start_date, end_date, variables)

df_weather.columns = ['temperature', 'dni', 'dhi', 'ghi']

df_weather

In [ ]:
df_weather.to_csv(os.path.join(raw_data_path, 'weather', 'weather_data_open-meteo.csv'))




# Data Cleaning, Merging

The goal here is the align the timesteps of each of the prepared dataframes and then save it in one common h5py file, to be imported in 'modeling.ipynb'

In [5]:
# importing / loading all data (saved in the above data prep cells)

# community load in 1min
df_load_15min = pd.read_hdf(os.path.join(raw_data_path, 'load', "df_load.h5"), key='15min/per_household_load').tz_localize(None)

#pv: power and meta, in 1min and 15min resolution
df_pv = pd.read_csv(os.path.join(raw_data_path, 'pv', 'filtered_pv_power_measurements_ac_2.csv'), index_col=0, parse_dates=True).tz_localize(None)
df_meta = pd.read_csv(os.path.join(raw_data_path, 'pv', 'metadata_with_lat_lng.csv'))

# weather data in 15 min resolution
# remember that you can get the data openly from https://open-meteo.com/en (although in hourly resolution)
df_weather = pd.read_csv(os.path.join(raw_data_path, 'weather', 'weather_data_open-meteo.csv'), index_col=0, parse_dates=True).tz_localize(None)



In [6]:
threshold = 0.95

df_load_15min_cleaned, days_to_drop = drop_days_with_nans_advances(df_load_15min, threshold)
df_dropped_days = dropped_days_plotted(df_load_15min, days_to_drop)
df_load_15min_cleaned.index = df_load_15min_cleaned.index - pd.DateOffset(years=4)
df_load_15min_cleaned = df_load_15min_cleaned.ffill(axis=1)



Dropped 116 days with more than 5.000000000000004% of the rows had 95.0% NaNs


In [7]:
df_pv = df_pv.ffill(axis=1)

In [8]:
df_all = pd.merge(df_load_15min_cleaned, df_pv, left_index=True, right_index=True, how='inner')
df_all = pd.merge(df_all, df_weather, left_index=True, right_index=True, how='left')
df_all.interpolate(method='linear', inplace=True, limit=60)

### Add time-of-use prices

In [9]:
# insert tou tariff depending on the time of the day:
# 0.08€/kWh from 00:00 to 06:00
# 0.20€/kWh from 06:00 to 08:00
# 0.08€/kWh from 08:00 to 18:00
# 0.20€/kWh from 18:00 to 24:00

df_all['tou'] = 0.08
df_all.loc[(df_all.index.hour >= 6) & (df_all.index.hour < 8), 'tou'] = 0.20
df_all.loc[(df_all.index.hour >= 18) & (df_all.index.hour < 24), 'tou'] = 0.20
df_all.loc[(df_all.index.hour >= 8) & (df_all.index.hour < 18), 'tou'] = 0.08


In [10]:
df_all = df_all.tz_localize(None)

In [11]:

freq = '15min'

ts_all = TimeSeries.from_dataframe(df_all, freq=freq)
ts_list = extract_subseries(ts_all)
ts_list_reviewed, _ = review_subseries(ts_list,60*24)
len(ts_list_reviewed)




12

In [12]:
df_all = ts_list_reviewed[0].pd_dataframe()
for i in range(1,len(ts_list_reviewed)):
    df_all = pd.concat([df_all, ts_list_reviewed[i].pd_dataframe()], axis=0)


# Generating Net Load Profiles with the Energy Manager

In [17]:
from bin.hems import HomeEnergyManager
import torch
 

df_netload_optimized = pd.DataFrame()
df_netload_not_optimized = pd.DataFrame()
df_state_of_energy = pd.DataFrame()


for ts in ts_list_reviewed:

    df_ = ts.pd_dataframe()

    hem = HomeEnergyManager(df_.shape[0], 10*1000, 5*1000, 0.25)

    load = df_.filter(like='SFH')
    pv = df_.filter(like='ID')
    tariff = df_['tou']
    inital_soe = 0.0

    # # to torch tensors
    load = torch.tensor(df_.filter(like='SFH').values, dtype=torch.float32).T.unsqueeze(-1)
    pv = torch.tensor(df_.filter(like='ID').values, dtype=torch.float32).T.unsqueeze(-1)
    tariff = torch.tensor(df_['tou'].values, dtype=torch.float32).unsqueeze(-1)
    inital_soe = torch.tensor(inital_soe, dtype=torch.float32).unsqueeze(-1)

    # # run the optimization
    netload, state_of_energy = hem.layer(load, pv, tariff, inital_soe)
    netload = netload.squeeze().detach().numpy().T
    state_of_energy = state_of_energy.squeeze().detach().numpy().T
    df_state_of_energy = pd.concat([df_state_of_energy, pd.DataFrame(state_of_energy, index=df_.index)], axis=0)
    df_netload_optimized = pd.concat([df_netload_optimized, pd.DataFrame(netload, index=df_.index)], axis=0)
    netload_without_optimization = (load - pv).T.squeeze().detach().numpy()
    df_netload_not_optimized = pd.concat([df_netload_not_optimized, pd.DataFrame(netload_without_optimization, index=df_.index)], axis=0)


In [35]:
# compare optimized and not optimized net load, and plot soe

idx_start = 10000
idx_end = 20000

fig = px.line(df_netload_optimized.iloc[idx_start:idx_end,0], title="Net load optimized")

fig.add_scatter(x=df_netload_not_optimized.index[idx_start:idx_end], y=df_netload_not_optimized.iloc[idx_start:idx_end, 0], mode='lines', name='Net load not optimized')

fig.add_scatter(x=df_state_of_energy.index[idx_start:idx_end], y=df_state_of_energy.iloc[idx_start:idx_end, 0], mode='lines', name='State of energy')

# Saving the cleaned and processed data

In [31]:
resolutions = ['15min']
store = pd.HDFStore(os.path.join(clean_data_path, "data_net_load_forecasting.h5"))

for resolution in resolutions:

    store.put(f'{resolution}/loads', df_all.filter(like='SFH').resample(resolution).mean(), format='table')
    store.put(f'{resolution}/pvs', df_all.filter(like='ID').resample(resolution).mean(), format='table')
    store.put(f'{resolution}/net_load_optimized', df_netload_optimized.resample(resolution).mean(), format='table')
    store.put(f'{resolution}/net_load_not_optimized', df_netload_not_optimized.resample(resolution).mean(), format='table')
    store.put(f'{resolution}/state_of_energy', df_state_of_energy.resample(resolution).mean(), format='table')
    store.put(f'{resolution}/weather', df_all[['temperature', 'dni', 'dhi', 'ghi']].resample(resolution).mean(), format='table')

store.put('pv_metadata', df_meta, format='table')

store.close()
